# Imports

In [ ]:
import packages

from context.utils import typer as t
from context.infra.clients import logger

from toolkit.utils import utils
from toolkit.utils.utils import rp_print
from toolkit.utils.llm import main as utils_llm

import context.instances as inst
import context.consts as const
import context.settings.main as settings_main

from toolkit.llm.langchain.core import integration, utils as utils_lc
from toolkit.llm.langchain.data.indexing import (
    documents, document_loaders, text_splitters,
)
from toolkit.llm.langchain.data.persistence import retrievers
from toolkit.llm.langchain.execution import (
    runnables, graphs, tools, agents, tools as tools_lc
)
from toolkit.llm.langchain.models import (
    prompts as prompts_lc, llms, messages,
)

# Dev

In [ ]:
class Person(t.BaseModel):
	"""Information about a person."""

	# ^ Doc-string for the entity Person.
	# This doc-string is sent to the LLM as the description of the schema Person,
	# and it can help to improve extraction results.

	# Note that:
	# 1. Each field is an `optional` -- this allows the model to decline to extract it!
	# 2. Each field has a `description` -- this description is used by the LLM.
	# Having a good description can help improve extraction results.
	name: t.Optional[str] = t.Field(
	default=None, description="The name of the person"
	)
	hair_color: t.Optional[str] = t.Field(
		default=None, description="The color of the person's hair if known"
	)
	height_in_meters: t.Optional[str] = t.Field(
		default=None, description="Height measured in meters"
	)

class Group(t.BaseModel):
  # Creates a model so that we can extract multiple entities.
	people: t.List[Person]

prompt_tpl = prompts_lc.ChatPromptTemplate.from_messages([
	(
		"system",
		"You are an expert extraction algorithm. Only extract relevant information from the text. If you do not know the value of an attribute asked to extract, return null for the attribute's value.",
	),
	(
   	"human", "{user_input}"
  ),
])

llm_structured = inst.llm_main.with_structured_output(schema=Group)

user_input = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_tpl.invoke({"user_input": user_input})
response = llm_structured.invoke(prompt)

rp_print(response)